In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api

In [ ]:
from IPython.display import clear_output
# Create a Python script to start the Ollama API server in a separate thread
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
from IPython.display import clear_output

!ollama pull llama3.1
clear_output()

!pip install -U lightrag[ollama]
clear_output()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama  # Mise à jour de l'importation
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import PyPDFLoader  # Pour charger les fichiers PDF
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings  # À garder si nécessaire

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.0 MB/s eta 0:00:00


In [ ]:
import os
print(os.getcwd()) # prints your current working directory
!ls -l # lists content of your working directory, including files and their permissions.

/content
total 1912
-rw-r--r-- 1 root root 1953257 Nov 20 15:28 progress.json
drwxr-xr-x 1 root root    4096 Nov 18 14:23 sample_data


In [ ]:
# # Charger un fichier PDF
pdf_loader = PyPDFLoader(file_path="data2.pdf")
docs = pdf_loader.load()

# Découper les documents en morceaux plus petits pour éviter la surcharge de mémoire
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [ ]:
from typing import List, Dict, Optional

class Node:
    def __init__(self, id: str, type: str, properties: Dict):
        self.id = id
        self.type = type
        self.properties = properties

    def __repr__(self):
        return f"Node(id='{self.id}', type='{self.type}', properties={self.properties})"

class Relationship:
    def __init__(self, source: Node, target: Node, type: str, properties: Dict):
        self.source = source
        self.target = target
        self.type = type
        self.properties = properties

    def __repr__(self):
        return f"Relationship(source={self.source}, target={self.target}, type='{self.type}', properties={self.properties})"

class Document:
    def __init__(self, metadata: Dict, page_content: str):
        self.metadata = metadata
        self.page_content = page_content

    def __repr__(self):
        return f"Document(metadata={self.metadata}, page_content='{self.page_content[:50]}...')"

class GraphDocument:
    def __init__(self, nodes: List[Node], relationships: List[Relationship], source: Document):
        self.nodes = nodes
        self.relationships = relationships
        self.source = source

    def __repr__(self):
        return f"GraphDocument(nodes={self.nodes}, relationships={self.relationships}, source={self.source})"


In [ ]:
import json

# Utilisation du modèle ChatOllama comme avant
llm = ChatOllama(model="llama3.1", temperature=0, format="json")  # Utilisation du modèle mis à jour
llm_transformer = LLMGraphTransformer(llm=llm)




# Fonction pour convertir un objet Document ou GraphDocument en format sérialisable
def serialize_document(doc):
    if hasattr(doc, "to_dict"):  # Vérifie si l'objet a une méthode to_dict()
        return doc.to_dict()
    elif hasattr(doc, "__dict__"):  # Sérialisation via attributs
        return {key: serialize_document(value) if isinstance(value, (list, dict, object)) else value
                for key, value in doc.__dict__.items()}
    elif isinstance(doc, list):  # Si c'est une liste, itère sur les éléments
        return [serialize_document(item) for item in doc]
    elif isinstance(doc, dict):  # Si c'est un dictionnaire, sérialise les valeurs
        return {key: serialize_document(value) for key, value in doc.items()}
    else:
        return doc  # Retourne la valeur brute pour les types simples (int, str, etc.)

# Initialiser une liste pour stocker les documents transformés
all_graph_documents = []

# Traiter chaque chunk et sauvegarder après chaque étape
for i, doc in enumerate(documents):
    print(f"Traitement du chunk {i + 1} / {len(documents)}...")

    # Convertir le document en format graphique
    graph_document = llm_transformer.convert_to_graph_documents([doc])

    # Sérialiser les documents transformés
    serialized_graph_document = [serialize_document(gd) for gd in graph_document]
    all_graph_documents.extend(serialized_graph_document)

    # Sauvegarder les progrès dans un fichier JSON
    with open("progress.json", "w") as f:
        json.dump(all_graph_documents, f, indent=4)

    print(f"Chunk {i + 1} traité et sauvegardé. Total transformé : {len(all_graph_documents)}")

print(f"Traitement terminé. Nombre total de documents transformés : {len(all_graph_documents)}")


In [ ]:
with open("progress.json", "r") as f:
    serialized_documents = json.load(f)

# Reconstitution des objets en format GraphDocument

# Recréer les objets GraphDocument
graph_documents = [
    GraphDocument(**doc) if isinstance(doc, dict) else doc
    for doc in serialized_documents
]

In [ ]:
graph_documents[0]

{'nodes': [],
 'relationships': [],
 'source': {'id': None,
  'metadata': {'source': 'data2.pdf', 'page': 0},
  'page_content': '1774 BULLETIN OFFICIEL Nº 7222 – 30 moharrem 1445 (17-8-2023) \n \n \n \nArrêté du ministre délégué auprès de la ministre de l’économie et des finances, chargé du budget n° 1692 -23 du 4 hija 1444  (23 juin 2023)',
  'type': 'Document'}}

In [ ]:
import json


def transform_json_to_graph(json_data: List[Dict]) -> List[GraphDocument]:
    graph_documents = []

    for item in json_data:
        nodes = [Node(id=node['id'], type=node['type'], properties=node.get('properties', {}))
                 for node in item.get('nodes', [])]

        relationships = [
            Relationship(
                source=Node(id=rel['source']['id'], type=rel['source']['type'], properties=rel['source'].get('properties', {})),
                target=Node(id=rel['target']['id'], type=rel['target']['type'], properties=rel['target'].get('properties', {})),
                type=rel['type'],
                properties=rel.get('properties', {})
            ) for rel in item.get('relationships', [])
        ]

        # Vérifier si nodes ou relationships ne sont pas vides
        if not nodes and not relationships:
            continue  # Ignorer cet item

        source_document = Document(
            metadata=item['source']['metadata'],
            page_content=item['source']['page_content']
        )

        graph_documents.append(GraphDocument(nodes=nodes, relationships=relationships, source=source_document))

    return graph_documents

# Charger le fichier progress.json
with open("progress.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Transformer en GraphDocuments
graph_documents = transform_json_to_graph(json_data)

# Vérifier la sortie
print(f"Nombre de documents transformés : {len(graph_documents)}")
print("Exemple d'un document transformé :")
if graph_documents:
  graph_documents[0]


Nombre de documents transformés : 669
Exemple d'un document transformé :


In [ ]:
graph_documents[0]


GraphDocument(nodes=[Node(id='dahir n° 1 -07-129', type='Decree', properties={}), Node(id='loi n° 53 -05', type='Law', properties={})], relationships=[Relationship(source=Node(id='loi n° 53 -05', type='Law', properties={}), target=Node(id='dahir n° 1 -07-129', type='Decree', properties={}), type='RELATED_TO', properties={})], source=Document(metadata={'source': 'data2.pdf', 'page': 0}, page_content='Vu la loi n° 53 -05 relative à l’échange électroni...'))

In [ ]:
!pip install --upgrade langchain
clear_output()

In [ ]:
os.environ["NEO4J_URI"] = 'neo4j+s://6c05ca87.databases.neo4j.io'
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "rWJH4DIO0X6a0iHG7844lXj2fFmus0I5NetFqdnGeYo"

In [ ]:
!ollama pull mxbai-embed-large
clear_output()

In [ ]:
from neo4j import GraphDatabase
from langchain.embeddings import OllamaEmbeddings
from langchain.graphs import Neo4jGraph
import os

# Set up Neo4j connection using environment variables
uri = os.environ.get("NEO4J_URI")  # Ensure these environment variables are set
user = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password))



# Set up Neo4jGraph and embeddings
graph = Neo4jGraph(url=uri, username=user, password=password)

#
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# Assuming you have a valid embeddings model (e.g., OllamaEmbeddings)
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Sample usage of graph (without Neo4jVector)
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

<ipython-input-17-c60880a25494>:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large")
ERROR:neo4j.io:[#B1F4]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.78.243.29', 7687)) (ResolvedIPv4Address(('34.78.243.29', 7687))): OSError('No data')
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information


In [ ]:
driver = GraphDatabase.driver(
        uri = os.environ.get('NEO4J_URI'),
        auth = (os.environ.get('NEO4J_USERNAME'), os.environ.get('NEO4J_PASSWORD')))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id`
    FOR (n:__Entity__)
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


In [ ]:
def showGraph() :
    driver =GraphDatabase.driver(
    uri = os. environ ["NEO4J_URI" ],
    auth =(os.environ ["NEO4J_USERNAME" ],
    os. environ ["NEO4J_PASSWORD" ]))
    session = driver.session()
    widget = GraphWidget (graph = session.run("MATCH (s)-[r: !MENTIONS] ->(t) RETURN s,r,t") . graph())
    widget.node_label_mapping = 'id'
    return widget
showGraph ()

In [ ]:
from pydantic import BaseModel, Field, ValidationError
import json


class Entities(BaseModel):
    """Identifying information about entities."""
    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that appear in the text",
    )

# Placeholder function to remove special characters for Lucene-compatible queries
def remove_lucene_chars(input_text: str) -> str:
    # Define characters to remove as needed (e.g., special symbols)
    lucene_special_chars = "!@#$%^&*()[]{}|\\;:'\"<>,.?/"
    return ''.join(char for char in input_text if char not in lucene_special_chars)

# Generates a full-text query for Lucene-based search compatibility
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()

# Extracts entities from text using the LLM
def extract_entities_from_text(question: str) -> Entities:
    # Format the prompt explicitly to ask for a JSON response
    formatted_prompt = (
        "You are extracting organization and person entities from the text. "
        "Extract entities from the following input and return them in JSON format. "
        "Use the format: {{'names': [...]}}. Input: {question}"
    ).format(question=question)

    # Ensure `llm` is initialized and invoke it with the prompt
    llm_response = llm.invoke(formatted_prompt)

    # Parse the response as JSON and validate it against the Entities model
    try:
        response_data = json.loads(llm_response.content)  # Parse the JSON response

        # Handle nested structure if 'entities' is the outer key
        if 'entities' in response_data and 'names' in response_data['entities']:
            response_data = response_data['entities']  # Extract only the inner 'names' dictionary

        entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model
        print(f"Extracted Entities: {entities}")
        return entities
    except (json.JSONDecodeError, ValidationError) as e:
        print("Error parsing response:", e)
        return Entities(names=[])

# Full-text index query and graph retrieval function
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned in the question
    """
    result = ""
    # Step 1: Extract entities from the question using LLM
    entities = extract_entities_from_text(question)

    # Step 2: Retrieve graph data based on each entity
    for entity in entities.names:
        full_text_query = generate_full_text_query(entity)
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": full_text_query},
        )
        # Process and format the graph retrieval output
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
llm = ChatOllama(model="llama3.1", temperature=0, format="json")  # Utilisation du modèle mis à jour


<ipython-input-20-0f415b4df970>:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1", temperature=0, format="json")  # Utilisation du modèle mis à jour


In [ ]:
print(graph_retriever("Who are candidature and architecte de l'administration?"))


Extracted Entities: names=['candidature', 'architecte']
Generated Query: candidature~2


Generated Query: architecte~2
f6e2c9a079b3ce5e9194145086c9f653 - MENTIONS -> candidature
08162371cb61ad7d936dcff76efca250 - MENTIONS -> candidaturesc1d755a8674268bd220836cfab7a3713 - MENTIONS -> architects
1bd94a7ed285de3f393235b971e3620c - MENTIONS -> architects
5c8743a8379e4129f3d09ec6c0fe85bd - MENTIONS -> architecte
182361ff8e5cf2cda19de40d86a1b601 - MENTIONS -> architecte
1941c437fa0b52a72f51cae0225c6d59 - MENTIONS -> architecte
7aad421d5b6e3c1f4bc990cb1cb7f497 - MENTIONS -> architecte
c4f5e1e0a9be3460ed1e22642ac95f29 - MENTIONS -> architecte
83752e2598141056294e18178f15d870 - MENTIONS -> architecte


In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]

    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data


In [ ]:
full_retriever("c'est quoi la relation entre jury et architects?")

Extracted Entities: names=['jury', 'architects']
Generated Query: jury~2
Generated Query: architects~2


"Graph data:\ne07ad1c4a7373e1a82cead96a93d56ef - MENTIONS -> Jury\nae3270dd1167e3b0169ed5e6aeb01cad - MENTIONS -> jury\n90bb289fc2ed5077292d3800832fc796 - MENTIONS -> jury\n729083f5f78f134693a58fa65d4aa8b0 - MENTIONS -> jury\n5164e92d49877a26b7589c4f4f860219 - MENTIONS -> jury\n08ad51cc7d7163f330af16b488293e15 - MENTIONS -> jury\n099e2aab90d36fe2336b8b32d8d45b1d - MENTIONS -> jury\n89886623b034cbc1097f103cee51424f - MENTIONS -> jury\n262554444647d62a40356d2f3e637355 - MENTIONS -> jury\nf8f7eadfd72210e3c964fecfc1337d68 - MENTIONS -> jury\n32f589a16bda2e1f7c89c0991ce6ea80 - MENTIONS -> jury\n0d6a3e4cbfde2768f20c4e20aaf24143 - MENTIONS -> jury\n6cb25059752b35a344bc3cfd2727a4e3 - MENTIONS -> jury\nc1d755a8674268bd220836cfab7a3713 - MENTIONS -> jury\n4b747a5c15ce0c69d91c002cf8ed203f - MENTIONS -> jury\n1bd94a7ed285de3f393235b971e3620c - MENTIONS -> juryc1d755a8674268bd220836cfab7a3713 - MENTIONS -> architects\n1bd94a7ed285de3f393235b971e3620c - MENTIONS -> architects\n5c8743a8379e4129f3d09e

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

### Q & A

In [ ]:
import logging

# Set up logging to suppress warnings from Neo4j
logging.getLogger("neo4j").setLevel(logging.ERROR)


In [ ]:
chain.invoke(input="c'est quoi la relation entre jury et architects?")

Extracted Entities: names=['jury', 'architects']
Generated Query: jury~2
Generated Query: architects~2


'{ "MENTIONS" : "architects", "TO" : "jury"}'

In [ ]:
import time
time.sleep(100)